## Preamble

Run these if notebook is in a new environment, otherwise just skip to `Import modules`

#### Boot cell

Skip if `module_update.py` is in same folder as notebook, next cell downloads this file which in turn will download other files.

In [139]:
!curl -s "https://raw.githubusercontent.com/Yoonsen/Modules/master/module_update.py" > "module_update.py"

#### Get essentials

Import text and graph modules for analysis, and css for display

In [140]:
from module_update import update, css

In [141]:
css()

# Import modules

Aktiver cellen nedenfor for å få alt til å virke.

In [142]:
update('nbtext', overwrite=True)
import nbtext as nb
from IPython.display import HTML
import requests
from bs4 import BeautifulSoup


def mods_digibok(urn, kind='marcxml'):
    """find information about serial number urn (or full urn) for books using NB API. 
    Parameter 'kind' is one of mods, marcxml or dublincore """
    
    if not str(urn).startswith('URN'):
        urnstr = "URN:NBN:no-nb_digibok_{urn}".format(urn=urn)
    r = requests.get("https://api.nb.no:443/catalog/v1/metadata/{ident}/{kind}".format(ident=urnstr, kind=kind))
    return r.text

def translator(urn):
    """Finds the translator for a book"""
    item = BeautifulSoup(mods_digibok(urn, kind='mods'), 'lxml')
    trans = []
    navn = item.find_all('name', {'type':'personal'})
    for i in navn:

        try:
            if i.roleterm.text == 'trl':
                trans.append(i.namepart.text)
        except:
            'nn'
    return ';'.join(trans)
    

Updated file `D:\Documents\GitHub\Oversettelse\nbtext.py`

## Build corpus

A corpus definition is a set of URNs. Those can be obtained in various ways and stored as text or in MS-Word file.


### Define using metadata 

Using author name, titles, year, dewey, subject and gender. Check out [Webdewey](http://deweysearchno.pansoft.de/webdeweysearch/index.html) for more descriptions.

In [143]:
korpus = nb.get_urn({
                     'year':1950, 
                     'next':2,
                     'trans':'eng',
                     'limit':300})
len(korpus)

300

In [144]:
korpus_nob = nb.get_urn({
                     'year':1950, 
                     'next':2,
                     'trans':'',
                     'limit':300})
len(korpus)

300

# Henter bøkene med ord

Her defineres korpuset som en liste over bøker, med informasjon om oversettere. Er korpuset stort (flere hundre) vil det ta noen minutter å hente alle oversetterne, så det kan være lurt å kutte ned størrelsen litt. Inntil vi får på plass en rask måte å gjøre det på, får det her duge. Det er bare å smøre seg med tålmodighet.

In [145]:
rammen = nb.frame(nb.metadata(korpus), ['urn', 'forfatter','år','tittel','språk','forlag', 'original' ]).sort_values(by='tittel')
rammen = rammen.set_index('urn')
rammen.index = rammen.index.astype(str)


book_counts = nb.frame(nb.book_count([x[0] for x in korpus]), ['antall_ord'])
book_counts.index = book_counts.index.astype(str)
book_counts.antall_ord = book_counts.antall_ord.astype(int)

print('Telling ferdig')

# sammenslåing
rammen = rammen.join(book_counts)
rammen = rammen[['forfatter', 'år', 'tittel', 'antall_ord', 'original']]
rammen.head()

Telling ferdig


,forfatter,år,tittel,antall_ord,original
urn,,,,,
2011031720018,,1950,"""Gud er sanndru""",106232.0,eng
2016060848216,"Orwell, George",1950,1984,118115.0,eng
2012022305002,"Bromfield, Louis",1952,24 timer,113596.0,eng
2011111108159,"Barber, D. H.",1951,Afrika i kaki,37382.0,eng
2007012601033,"Seaver, George",1950,Albert Schweitzer,71866.0,eng


## Se på hele korpuset under ett som HTML



In [146]:
HTML(rammen.sort_values(by = 'forfatter').to_html())

,forfatter,år,tittel,antall_ord,original
urn,,,,,
2011031720018,,1950,"""Gud er sanndru""",106232.0,eng
2011110708036,,1951,De tre bjørnene,811.0,eng
2008042800003,,1950,Verdenserklæringen om menneskerettigheter,10158.0,eng
2016110248029,"Abrahams, Peter",1951,Røde hjulspor,115901.0,eng
2016062048127,"Albrand, Martha",1950,andre kvinnen,107377.0,eng
2010070505015,"Alcott, Louisa May",1951,Småfrøkner,43664.0,eng
2008050504059,"Altsheler, Joseph A.",1952,siste høvding,48468.0,eng
2011112308150,Ariane,1952,Petter Harepus,1070.0,eng
2013021513004,"Babcock, Dwight V.",1950,Hannas hobby,119073.0,eng


## Search corpus for concordances

In [147]:
from random import sample

In [148]:
nb.get_urnkonk('gråtende', {'urns':sample(korpus, 50)})

"Giftbanden som truet London, Fletcher, J.S., 1950","kvinnelige tjenere , — alle",gråtende,.
"Hagars sønn, Caine, Hall, 1952",! » sa arrestanten med,gråtende,"stemme . « Paul ,"
"Hagars sønn, Caine, Hall, 1952",var som en engel gikk,gråtende,forbi .
"Hagars sønn, Caine, Hall, 1952",Greta kom ut med en,gråtende,liten gutt på armen .
"Kjærlighet over alle grenser, Galsworthy, John, 1951",av seg da Betty kom,gråtende,inn til henne . «
"Kjærlighet over alle grenser, Galsworthy, John, 1951",Hvis hun tenkte på det,gråtende,"barnet , visste hun at"
"Natten og byen, Kersh, Gerald, 1951","som du er nå ,",gråtende,over en fordømt flaske .
"tjenende ånd, Wodehouse, P.G., 1952",Å omfavne en,gråtende,ung pike som sitter bak
"Vrakgods, Valtin, Jan, 1951",tre stålklørne opp foran det,gråtende,ansikt . Kurz loftet hendene
"Arven, Croft-Cooke, Rupert, 1950",begravd ? » kom det,gråtende,fra Maud . « Men
"Lev livet uten bekymringer, Carnegie, Dale, 1950",fant en av offiserene ham,gråtende,", og spurte hva som"


# Definer teksten

Hen teksten - kan ta noe minutter. Kan lagres for senere bruk.

In [149]:
TEXT = nb.get_corpus_text(rammen.index)

In [150]:
with open ('korpus_file.csv','w',encoding="utf-8") as f:
    TEXT.to_csv(path_or_buf=f)

In [151]:
TEXT_NOB = nb.get_corpus_text(nb.frame(korpus_nob)[0])

#### convertet to md
import pandas as pd
with open ('korpus_file.csv','r', encoding="utf-8") as f:
    text = pd.read_csv(f)

In [152]:
TEXT.sort_values(by=TEXT.columns[0], ascending=False)[:10]

,2011031720018,2016060848216,2012022305002,2011111108159,2007012601033,2008042804073,2011022320092,2011022320090,2013013108105,2008042804098,...,2013092308000,2008050504059,2008051404099,2010070506039,2011102108219,2008051404091,2008041804072,2008042504034,2011050420019,2007020201061
",",5892.0,5321.0,4845.0,1877.0,3080.0,2331.0,3169.0,3659.0,2160.0,8553.0,...,16.0,2688.0,2750.0,11509.0,2529.0,2723.0,2332.0,3851.0,1974.0,979.0
.,2958.0,5994.0,5258.0,1490.0,2710.0,3605.0,3425.0,6143.0,1667.0,9072.0,...,30.0,2491.0,3495.0,13372.0,2967.0,5012.0,3096.0,5058.0,4366.0,954.0
og,2681.0,3024.0,4744.0,1175.0,1623.0,2152.0,2200.0,1944.0,1565.0,4857.0,...,16.0,1654.0,1951.0,6095.0,2065.0,1433.0,1227.0,3250.0,1352.0,434.0
:,2430.0,155.0,472.0,38.0,179.0,200.0,28.0,70.0,41.0,262.0,...,1.0,70.0,114.0,246.0,32.0,52.0,248.0,78.0,85.0,66.0
som,1897.0,2104.0,1650.0,616.0,1158.0,800.0,726.0,1009.0,414.0,2748.0,...,4.0,438.0,897.0,2574.0,611.0,927.0,699.0,935.0,648.0,283.0
i,1715.0,2252.0,1765.0,835.0,1729.0,1235.0,1121.0,1356.0,738.0,3385.0,...,5.0,723.0,1097.0,3914.0,1061.0,1173.0,771.0,1734.0,987.0,359.0
er,1661.0,955.0,573.0,195.0,829.0,262.0,711.0,1010.0,251.0,5070.0,...,4.0,252.0,424.0,1776.0,198.0,664.0,434.0,919.0,536.0,520.0
det,1485.0,2603.0,1961.0,639.0,1090.0,1221.0,1495.0,2185.0,680.0,4120.0,...,4.0,1053.0,1029.0,3279.0,745.0,1274.0,925.0,1600.0,921.0,377.0
til,1448.0,1369.0,1419.0,587.0,1161.0,1205.0,914.0,1372.0,512.0,3027.0,...,4.0,563.0,800.0,2255.0,931.0,930.0,709.0,1254.0,741.0,259.0
«,1328.0,151.0,112.0,37.0,625.0,94.0,42.0,1930.0,27.0,335.0,...,19.0,516.0,1446.0,105.0,656.0,2421.0,1320.0,2219.0,22.0,91.0


### Starter med listen over mulige partisipper

In [153]:
poss_part = list ({x for x in TEXT.index if x.endswith('ende')})

### Henter ut de 100 mest frekvente

In [154]:
N = 100
test_words = list(TEXT.loc[poss_part].sum(axis=1).sort_values(ascending=False)[:N].index)

In [155]:
variants = dict()
for w in test_words:
    variants[w] = nb.word_paradigm(w)

In [156]:
forms = dict()
for w in test_words:
    forms[w] = nb.word_form(w)

In [157]:
nb.frame(variants)

,0,1,2,3
stående,"[adj, [stående, stått, ståtte]]","[verb, [sto, stod, stå, ståes, står, stås, stå...",None,None
sende,"[adj, [sendende, sendt, sendte]]","[verb, [send, sende, sender, sendes, sendt, se...",None,None
levende,"[adj, [levd, levde, levede, levende, levet, le...","[verb, [lev, levd, levde, leve, lever, leves, ...",None,None
hende,"[adj, [hendende, hendt, hendte, hende]]","[subs, [henda, hende, hendene, hender, hendet]]","[verb, [hend, hende, hender, hendes, hendt, he...",None
rasende,"[adj, [rasa, rasende, rast, raste]]","[adv, [rasende]]","[verb, [ras, rasa, rase, raser, rases, rast, r...",None
ende,"[adj, [endende, endt, endte]]","[adv, [ende]]","[subs, [ende, enden, endene, ender]]","[verb, [end, ende, ender, endes, endt, endte]]"
strålende,"[adj, [strålende, strålt, strålte]]","[verb, [strål, stråle, stråler, stråles, strål...",None,None
sittende,"[adj, [sitne, sitten, sittende, sittet]]","[verb, [satt, sitt, sitte, sitter, sittes, sit...",None,None
liggende,"[adj, [liggen, liggende, ligget, ligne]]","[verb, [ligg, ligge, ligger, ligges, ligget, l...",None,None
vende,"[adj, [vendende, vendt, vendte]]","[subs, [venda, vende, venden, vendene, vender]]","[verb, [vend, vende, vender, vendes, vendt, ve...",None


## Velg former

Plukk ut partisippformene, dersom formen opptrer både som adjektiv og som verb, ellers ut. Nedenfor plukkes ut de alternative verbformene som kan sammenlignes med partisippen.

Her benyttes en enkelt heuristikk. Partisippformen må være oppført både som et adjektiv og et verb, i tillegg må ikke formen selv være blandt verbene.

In [158]:
participles = dict()
for w in variants:
    # finn POS for alle variantene
    kinds = [x[0] for x in variants[w]]
    
    # sjekk at både adj og verb er blant dem
    if 'adj' and 'verb' in kinds:
        
        # Ta ut de som har formen blant verbene (partisippet er ikke verd i NO)
        F = [k[1] for k in variants[w] if k[0] == 'verb' and not w in k[1] ]
        if F != []:
            participles[w] = F

## Se på resultatet

In [159]:
participles

{'stående': [['sto', 'stod', 'stå', 'ståes', 'står', 'stås', 'stått']],
 'levende': [['lev', 'levd', 'levde', 'leve', 'lever', 'leves', 'levet']],
 'rasende': [['ras', 'rasa', 'rase', 'raser', 'rases', 'rast', 'raste']],
 'strålende': [['strål', 'stråle', 'stråler', 'stråles', 'strålt', 'strålte']],
 'sittende': [['satt', 'sitt', 'sitte', 'sitter', 'sittes', 'sittet']],
 'liggende': [['ligg', 'ligge', 'ligger', 'ligges', 'ligget', 'lå', 'låg']],
 'følgende': [['følg',
   'følgje',
   'følgjer',
   'følgjes',
   'følgt',
   'følgte',
   'fulgt',
   'fulgte',
   'følge',
   'følger',
   'følges']],
 'smilende': [['smil', 'smile', 'smiler', 'smiles', 'smilt', 'smilte']],
 'skinnende': [['skein',
   'skinn',
   'skinne',
   'skinner',
   'skinnes',
   'skint',
   'skinte']],
 'skjelvende': [['skalv',
   'skjelv',
   'skjelve',
   'skjelver',
   'skjelves',
   'skjelvet']],
 'brennende': [['brenn',
   'brenne',
   'brenner',
   'brennes',
   'brent',
   'brente',
   'brann',
   'brant']],
 

Nå lager vi en ny statistikk der alle ordene telles opp fra datarammen for sammenligning

In [160]:
counts = dict()
for w in participles:
    counts[w] = TEXT.loc[list(set(participles[w][0]) & set(TEXT.index))].sum(axis=1).sum()

In [161]:
counts_nob = dict()
for w in participles:
    counts_nob[w] = TEXT_NOB.loc[list(set(participles[w][0]) & set(TEXT_NOB.index))].sum(axis=1).sum()

In [162]:
counts_nob

{'stående': 20132.0,
 'levende': 5588.0,
 'rasende': 402.0,
 'strålende': 463.0,
 'sittende': 16136.0,
 'liggende': 14831.0,
 'følgende': 5755.0,
 'smilende': 3158.0,
 'skinnende': 1080.0,
 'skjelvende': 598.0,
 'brennende': 1725.0,
 'utseende': 101.0,
 'gående': 39876.0,
 'spennende': 882.0,
 'truende': 3178.0,
 'lignende': 761.0,
 'avgjørende': 1169.0,
 'passende': 2233.0,
 'vedkommende': 127.0,
 'forbausende': 312.0,
 'hengende': 1974.0,
 'løpende': 2614.0,
 'glimrende': 20.0,
 'utelukkende': 266.0,
 'spørrende': 4315.0,
 'angående': 440.0,
 'ledende': 2095.0,
 'oppmuntrende': 108.0,
 'beroligende': 89.0,
 'glødende': 213.0,
 'overraskende': 176.0,
 'triumferende': 49.0,
 'imponerende': 103.0,
 'overveldende': 64.0,
 'flytende': 609.0,
 'lysende': 5233.0,
 'gjennomtrengende': 19.0,
 'tilfredsstillende': 223.0,
 'anstrengende': 139.0,
 'skingrende': 30.0,
 'omfattende': 1177.0,
 'døende': 3848.0,
 'liknende': 515.0,
 'flammende': 296.0,
 'ridende': 844.0,
 'gjeldende': 7162.0,
 'reis

In [163]:
counts

{'stående': 36923.0,
 'levende': 8255.0,
 'rasende': 1132.0,
 'strålende': 904.0,
 'sittende': 35308.0,
 'liggende': 22544.0,
 'følgende': 11687.0,
 'smilende': 11579.0,
 'skinnende': 1789.0,
 'skjelvende': 2135.0,
 'brennende': 2949.0,
 'utseende': 82.0,
 'gående': 88301.0,
 'spennende': 2025.0,
 'truende': 1680.0,
 'lignende': 1861.0,
 'avgjørende': 1435.0,
 'passende': 5642.0,
 'vedkommende': 134.0,
 'forbausende': 2678.0,
 'hengende': 5147.0,
 'løpende': 7930.0,
 'glimrende': 21.0,
 'utelukkende': 271.0,
 'spørrende': 24449.0,
 'angående': 958.0,
 'ledende': 2996.0,
 'oppmuntrende': 604.0,
 'beroligende': 553.0,
 'glødende': 567.0,
 'overraskende': 1302.0,
 'triumferende': 123.0,
 'imponerende': 407.0,
 'overveldende': 408.0,
 'flytende': 955.0,
 'lysende': 12891.0,
 'gjennomtrengende': 51.0,
 'tilfredsstillende': 422.0,
 'anstrengende': 608.0,
 'skingrende': 55.0,
 'omfattende': 542.0,
 'døende': 9231.0,
 'liknende': 699.0,
 'flammende': 862.0,
 'ridende': 2955.0,
 'gjeldende': 57

# Legg til opptellinger for partisippene

In [164]:
count_frame = dict()
for x in counts:
    count_frame[x] = {'verb_paradigm': counts[x], 'participle':TEXT.loc[x].sum()}

In [165]:
count_frame_nob = dict()
for x in counts_nob:
    count_frame_nob[x] = {'verb_paradigm': counts_nob[x], 'participle':TEXT_NOB.loc[x].sum()}

In [166]:
count_frame_nob

{'stående': {'verb_paradigm': 20132.0, 'participle': 1275.0},
 'levende': {'verb_paradigm': 5588.0, 'participle': 1652.0},
 'rasende': {'verb_paradigm': 402.0, 'participle': 357.0},
 'strålende': {'verb_paradigm': 463.0, 'participle': 382.0},
 'sittende': {'verb_paradigm': 16136.0, 'participle': 635.0},
 'liggende': {'verb_paradigm': 14831.0, 'participle': 738.0},
 'følgende': {'verb_paradigm': 5755.0, 'participle': 2581.0},
 'smilende': {'verb_paradigm': 3158.0, 'participle': 263.0},
 'skinnende': {'verb_paradigm': 1080.0, 'participle': 195.0},
 'skjelvende': {'verb_paradigm': 598.0, 'participle': 137.0},
 'brennende': {'verb_paradigm': 1725.0, 'participle': 254.0},
 'utseende': {'verb_paradigm': 101.0, 'participle': 356.0},
 'gående': {'verb_paradigm': 39876.0, 'participle': 360.0},
 'spennende': {'verb_paradigm': 882.0, 'participle': 162.0},
 'truende': {'verb_paradigm': 3178.0, 'participle': 172.0},
 'lignende': {'verb_paradigm': 761.0, 'participle': 860.0},
 'avgjørende': {'verb_p

In [167]:
COUNTS = nb.frame(count_frame)

In [168]:
COUNTS_NOB = nb.frame(count_frame_nob)

Legg en ratio til

In [169]:
COUNTS['ratio'] = COUNTS['participle']/(COUNTS['participle']+COUNTS['verb_paradigm'])

In [170]:
COUNTS_NOB['ratio'] = COUNTS_NOB['participle']/(COUNTS_NOB['participle']+COUNTS_NOB['verb_paradigm'])

# Sortér på ratio

Her ser vi klart at det kanskje ikke er ekte partisipper på topp, mer typiske ord i partisippform.

De mest interessante ordene (tror jeg) er på bunnen her, som 'smilende', 'løpende', 'liggende', 'spørrende'. Og den ratioen kan sammenlignes med norsk sett.

In [171]:
COUNTS.sort_values(by='ratio', ascending=False)

,verb_paradigm,participle,ratio
glimrende,21.0,550.0,0.963222
sjarmerende,14.0,251.0,0.947170
utseende,82.0,782.0,0.905093
gjennomtrengende,51.0,366.0,0.877698
tilsvarende,44.0,292.0,0.869048
skingrende,55.0,330.0,0.857143
vedkommende,134.0,626.0,0.823684
fortryllende,59.0,222.0,0.790036
triumferende,123.0,412.0,0.770093
rasende,1132.0,2345.0,0.674432


In [172]:
COUNTS_NOB.sort_values(by='ratio', ascending=False)

,verb_paradigm,participle,ratio
vedkommende,127.0,2978.0,0.959098
tilsvarende,70.0,957.0,0.931840
glimrende,20.0,178.0,0.898990
fortryllende,6.0,35.0,0.853659
sjarmerende,11.0,64.0,0.853333
utseende,101.0,356.0,0.778993
tilfredsstillende,223.0,474.0,0.680057
utfordrende,27.0,49.0,0.644737
overveldende,64.0,115.0,0.642458
gjennomtrengende,19.0,34.0,0.641509


# Sammenligner ratio for oversatt mot NOB

In [173]:
COMP = COUNTS['ratio']/COUNTS_NOB['ratio']

In [174]:
HTML(nb.frame(COMP.sort_values(ascending=False)).to_html())

,ratio
leende,8.721100
truende,5.848702
ertende,4.640791
undersøkende,3.423775
trykkende,2.580386
granskende,2.400924
farende,2.291177
skinnende,2.275594
tiltalende,1.913563
inngående,1.807862
